In [1]:
import opensfm.reconstruction as rec
from opensfm import dataset
from opensfm import pysfm
from opensfm import tracking

import logging
from timeit import default_timer as timer
import numpy as np
import math
import time

logger = logging.getLogger(__name__)

In [2]:
def incremental_reconstruction(data, tracks_manager, foc_len):
    """Run the entire incremental reconstruction pipeline."""
    logger.info("Starting incremental reconstruction")
    report = {}
    chrono = rec.Chronometer()

    images = tracks_manager.get_shot_ids()
    
    if not data.reference_lla_exists():
        data.invent_reference_lla(images)

    remaining_images = set(images)
    camera_priors = data.load_camera_models()
    for i in camera_priors.keys():
        camera_priors[i].focal = foc_len
    gcp = data.load_ground_control_points()
    common_tracks = tracking.all_common_tracks(tracks_manager)
    reconstructions = []
    pairs = rec.compute_image_pairs(common_tracks, camera_priors, data)
    chrono.lap('compute_image_pairs')
    report['num_candidate_image_pairs'] = len(pairs)
    report['reconstructions'] = []
    
    t0 = time.time()
    
    for im1, im2 in pairs:
        if im1 in remaining_images and im2 in remaining_images:
            "Running boostrap and BA"
            rec_report = {}
            report['reconstructions'].append(rec_report)
            _, p1, p2 = common_tracks[im1, im2]
            reconstruction, graph_inliers, rec_report['bootstrap'] = rec.bootstrap_reconstruction(
                data, tracks_manager, camera_priors, im1, im2, p1, p2)

            if reconstruction:
                remaining_images.remove(im1)
                remaining_images.remove(im2)
                reconstruction, rec_report['grow'] = rec.grow_reconstruction(
                    data, tracks_manager, graph_inliers, reconstruction, remaining_images, camera_priors, gcp)
                reconstructions.append(reconstruction)
                reconstructions = sorted(reconstructions,
                                         key=lambda x: -len(x.shots))
                rec_report['stats'] = rec.compute_statistics(reconstruction, graph_inliers)
                logger.info(rec_report['stats'])
                
    t1 = time.time()
    
    print("time: ", t1-t0)

    for k, r in enumerate(reconstructions):
        logger.info("Reconstruction {}: {} images, {} points".format(
            k, len(r.shots), len(r.points)))
    logger.info("{} partial reconstructions in total.".format(
        len(reconstructions)))
    chrono.lap('compute_reconstructions')
    report['wall_times'] = dict(chrono.lap_times())
    report['not_reconstructed_images'] = list(remaining_images)
    return report, reconstructions

In [43]:
data = dataset.DataSet('data/iphone11pro_wide_office_30_parallax')
tracks_manager = data.load_tracks_manager()

# tracks_manager = pysfm.TracksManager.instanciate_from_file('/Users/jarrodhaas/torus-fast-track-dependencies/OpenSfM/data/gopro8_grass/tracks.csv')

In [42]:
tol = 0.01
init_focal = 0.8

t0 = time.time()

for i in range(15):

    report, reconstructions = incremental_reconstruction(data, tracks_manager, foc_len=init_focal)

    rec1 = reconstructions.pop()
    for i in rec1.cameras.keys():
        est_focal = rec1.cameras[i].focal

    if ((init_focal - est_focal) <= tol):
        print("Tolerance reached. Focal est: ", est_focal, "Iteration: ",i)
        t1 = time.time()
        print("Total time: ", t1-t0)
        break
    else:
        print("est focal: ", est_focal)
        init_focal = est_focal
        t1 = time.time()
        print("Total time: ", t1-t0)
        
    

2020-08-04 13:58:32,872 INFO: Starting incremental reconstruction
2020-08-04 13:58:36,167 INFO: Starting reconstruction with 0208.png and 0225.png
2020-08-04 13:58:36,203 INFO: Two-view reconstruction inliers: 1354 / 1384
2020-08-04 13:58:36,340 INFO: Triangulated: 1364
2020-08-04 13:58:36,359 DEBUG: Ceres Solver Report: Iterations: 5, Initial cost: 1.275642e+02, Final cost: 1.230574e+02, Termination: CONVERGENCE
2020-08-04 13:58:36,547 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 1.238982e+02, Final cost: 1.215318e+02, Termination: CONVERGENCE
2020-08-04 13:58:36,655 DEBUG: Ceres Solver Report: Iterations: 17, Initial cost: 2.446647e+02, Final cost: 2.200588e+02, Termination: CONVERGENCE
2020-08-04 13:58:36,659 INFO: Removed outliers: 2
2020-08-04 13:58:36,666 INFO: -------------------------------------------------------
2020-08-04 13:58:36,685 INFO: 0224.png resection inliers: 1187 / 1205
2020-08-04 13:58:36,714 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 2.3

2020-08-04 13:58:50,403 DEBUG: Ceres Solver Report: Iterations: 14, Initial cost: 4.794066e+03, Final cost: 4.554298e+03, Termination: CONVERGENCE
2020-08-04 13:58:52,690 DEBUG: Ceres Solver Report: Iterations: 17, Initial cost: 4.914976e+03, Final cost: 4.758110e+03, Termination: CONVERGENCE
2020-08-04 13:58:52,743 INFO: Removed outliers: 10
2020-08-04 13:58:52,750 INFO: -------------------------------------------------------
2020-08-04 13:58:52,778 INFO: 0189.png resection inliers: 1688 / 1906
2020-08-04 13:58:52,821 DEBUG: Ceres Solver Report: Iterations: 6, Initial cost: 1.105129e+03, Final cost: 1.052303e+03, Termination: CONVERGENCE
2020-08-04 13:58:52,822 INFO: Adding 0189.png to the reconstruction
2020-08-04 13:58:52,894 DEBUG: Local bundle sets: interior 14  boundary 0  other 0
2020-08-04 13:58:53,806 DEBUG: Ceres Solver Report: Iterations: 11, Initial cost: 5.227726e+03, Final cost: 5.090088e+03, Termination: NO_CONVERGENCE
2020-08-04 13:58:53,868 INFO: Removed outliers: 4
20

2020-08-04 13:59:12,974 INFO: Removed outliers: 1
2020-08-04 13:59:12,979 INFO: -------------------------------------------------------
2020-08-04 13:59:13,011 INFO: 0162.png resection inliers: 653 / 1459
2020-08-04 13:59:13,033 DEBUG: Ceres Solver Report: Iterations: 5, Initial cost: 1.140525e+03, Final cost: 1.116023e+03, Termination: CONVERGENCE
2020-08-04 13:59:13,034 INFO: Adding 0162.png to the reconstruction
2020-08-04 13:59:13,173 DEBUG: Local bundle sets: interior 26  boundary 0  other 0
2020-08-04 13:59:14,526 DEBUG: Ceres Solver Report: Iterations: 11, Initial cost: 9.572059e+03, Final cost: 9.477768e+03, Termination: NO_CONVERGENCE
2020-08-04 13:59:14,617 INFO: Removed outliers: 4
2020-08-04 13:59:14,622 INFO: -------------------------------------------------------
2020-08-04 13:59:14,655 INFO: 0159.png resection inliers: 599 / 1372
2020-08-04 13:59:14,671 DEBUG: Ceres Solver Report: Iterations: 6, Initial cost: 1.104125e+03, Final cost: 1.079796e+03, Termination: CONVERGEN

time:  52.169692039489746
est focal:  0.5375769194446509
Total time:  55.59014010429382


2020-08-04 13:59:32,604 INFO: Starting reconstruction with 0175.png and 0204.png
2020-08-04 13:59:32,627 INFO: Two-view reconstruction inliers: 1124 / 1143
2020-08-04 13:59:32,790 INFO: Triangulated: 1018
2020-08-04 13:59:32,807 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 3.544465e+01, Final cost: 3.482414e+01, Termination: CONVERGENCE
2020-08-04 13:59:32,984 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 3.491706e+01, Final cost: 3.463823e+01, Termination: CONVERGENCE
2020-08-04 13:59:33,050 DEBUG: Ceres Solver Report: Iterations: 13, Initial cost: 7.422283e+01, Final cost: 6.258332e+01, Termination: CONVERGENCE
2020-08-04 13:59:33,053 INFO: Removed outliers: 1
2020-08-04 13:59:33,060 INFO: -------------------------------------------------------
2020-08-04 13:59:33,075 INFO: 0177.png resection inliers: 858 / 869
2020-08-04 13:59:33,094 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 8.277240e+01, Final cost: 4.069081e+01, Termination: CONVERGENCE
2020-0

...

time:  51.1324200630188
est focal:  0.4667412816428425
Total time:  163.45712208747864


2020-08-04 14:01:20,865 INFO: Starting reconstruction with 0194.png and 0208.png
2020-08-04 14:01:20,897 INFO: Two-view reconstruction inliers: 1508 / 1531
2020-08-04 14:01:21,021 INFO: Triangulated: 947
2020-08-04 14:01:21,035 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 1.832996e+01, Final cost: 1.359594e+01, Termination: CONVERGENCE
2020-08-04 14:01:21,213 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 1.407714e+01, Final cost: 1.211220e+01, Termination: CONVERGENCE
2020-08-04 14:01:21,271 DEBUG: Ceres Solver Report: Iterations: 9, Initial cost: 3.250362e+01, Final cost: 2.311824e+01, Termination: CONVERGENCE
2020-08-04 14:01:21,275 INFO: Removed outliers: 0
2020-08-04 14:01:21,281 INFO: -------------------------------------------------------
2020-08-04 14:01:21,298 INFO: 0198.png resection inliers: 794 / 810
2020-08-04 14:01:21,315 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 3.635434e+01, Final cost: 2.783016e+01, Termination: CONVERGENCE
2020-08-

2020-08-04 14:01:32,289 DEBUG: Ceres Solver Report: Iterations: 12, Initial cost: 1.550067e+03, Final cost: 1.488284e+03, Termination: CONVERGENCE
2020-08-04 14:01:32,330 INFO: Removed outliers: 0
2020-08-04 14:01:32,337 INFO: -------------------------------------------------------
2020-08-04 14:01:32,367 INFO: 0164.png resection inliers: 1756 / 1878
2020-08-04 14:01:32,411 DEBUG: Ceres Solver Report: Iterations: 5, Initial cost: 2.557947e+02, Final cost: 1.964141e+02, Termination: CONVERGENCE
2020-08-04 14:01:32,412 INFO: Adding 0164.png to the reconstruction
2020-08-04 14:01:32,475 DEBUG: Local bundle sets: interior 14  boundary 0  other 0
2020-08-04 14:01:33,091 DEBUG: Ceres Solver Report: Iterations: 7, Initial cost: 1.731977e+03, Final cost: 1.651332e+03, Termination: CONVERGENCE
2020-08-04 14:01:33,141 INFO: Removed outliers: 0
2020-08-04 14:01:33,147 INFO: -------------------------------------------------------
2020-08-04 14:01:33,177 INFO: 0165.png resection inliers: 1918 / 201

2020-08-04 14:01:55,525 INFO: -------------------------------------------------------
2020-08-04 14:01:55,551 INFO: 0218.png resection inliers: 1270 / 1447
2020-08-04 14:01:55,586 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 1.523969e+02, Final cost: 1.356877e+02, Termination: CONVERGENCE
2020-08-04 14:01:55,587 INFO: Adding 0218.png to the reconstruction
2020-08-04 14:01:55,714 DEBUG: Local bundle sets: interior 26  boundary 0  other 0
2020-08-04 14:01:57,262 DEBUG: Ceres Solver Report: Iterations: 10, Initial cost: 3.765016e+03, Final cost: 3.708224e+03, Termination: CONVERGENCE
2020-08-04 14:01:57,368 INFO: Removed outliers: 1
2020-08-04 14:01:57,373 INFO: -------------------------------------------------------
2020-08-04 14:01:57,399 INFO: 0220.png resection inliers: 1269 / 1478
2020-08-04 14:01:57,442 DEBUG: Ceres Solver Report: Iterations: 5, Initial cost: 1.750952e+02, Final cost: 1.457388e+02, Termination: CONVERGENCE
2020-08-04 14:01:57,443 INFO: Adding 0220.png to

time:  54.025781869888306
Tolerance reached. Focal est:  0.4627679233101594 Iteration:  v2 unknown unknown 1920 1080 perspective 0
Total time:  222.16279006004333


In [44]:
camera_priors = data.load_camera_models()
for i in camera_priors.keys():
    img_width = camera_priors[i].width
    img_height = camera_priors[i].height

In [49]:
key = next(iter(camera_priors.keys()))
img_width = camera_priors[key].width


In [50]:
img_width

1920

In [30]:
from opensfm import io
# with open(data.profile_log(), 'a') as fout:
#             fout.write('reconstruct: {0}\n'.format(end - start))
data.save_reconstruction(reconstructions)
data.save_report(io.json_dumps(report), 'reconstruction.json')

34548

In [110]:
(1920-1153) / 1920

0.39947916666666666

In [31]:
.436 * 1920

837.12

# How are we calculating bearings?

In [16]:
camera_priors = data.load_camera_models()

In [152]:
common_tracks = tracking.all_common_tracks(tracks_manager)
_, p1, p2 = common_tracks['0004.png', '0030.png']

In [27]:
camera_priors['v2 unknown unknown 1920 1080 perspective 0'].get_K()

array([[0.8, 0. , 0. ],
       [0. , 0.8, 0. ],
       [0. , 0. , 1. ]])

In [158]:
K = camera_priors['v2 unknown unknown 1920 1080 perspective 0'].get_K_in_pixel_coordinates(1920,1080)

In [159]:
K

array([[1.632e+03, 0.000e+00, 9.600e+02],
       [0.000e+00, 1.632e+03, 5.400e+02],
       [0.000e+00, 0.000e+00, 1.000e+00]])

In [90]:
def sqrNorm(a):
    return a[0]**2 + a[1]**2

In [83]:
b1 = a[0] / (0.85) * 0.98956846

In [88]:
0.04465131 / 0.98956846 * 0.85

0.038353701673151545

In [148]:
point = np.array([ p1[0,0] , p1[0,1]])

inv_norm = 1.0 / ((sqrNorm(point) + 1.0)**0.5)

In [149]:
a2[0] * inv_norm

0.038258862027737504

In [150]:
inv_norm

0.9924304410101045

In [162]:
from opensfm import exif

In [ ]:
d = exif.extract_exif_from_file('data/office/images/0006.png')

In [250]:
0.5150781691074733* 1920

988.9500846863486

# Camera Metadata Test

In [2]:
import copy
import logging
import time

from opensfm import dataset
from opensfm import exif

In [3]:
logger = logging.getLogger(__name__)
logging.getLogger("exifread").setLevel(logging.WARNING)

In [4]:
def _extract_exif(image, data):
     # EXIF data in Image
    d = exif.extract_exif_from_file(data.open_image_file(image))

    # Image Height and Image Width
    if d['width'] <= 0 or not data.config['use_exif_size']:
        d['height'], d['width'] = data.image_size(image)

    d['camera'] = exif.camera_id(d)

    return d


start = time.time()
data = dataset.DataSet('data/office')

exif_overrides = {}
if data.exif_overrides_exists():
    exif_overrides = data.load_exif_overrides()

camera_models = {}
for image in data.images():
#     if data.exif_exists(image):
#         logging.info('Loading existing EXIF for {}'.format(image))
#         d = data.load_exif(image)
#     else:
        
    logging.info('Extracting EXIF for {}'.format(image))
    d = _extract_exif(image, data)

    if image in exif_overrides:
        
        d.update(exif_overrides[image])

    data.save_exif(image, d)

    if d['camera'] not in camera_models:
        
        camera = exif.camera_from_exif_metadata(d, data)
        camera_models[d['camera']] = camera
        break

# Override any camera specified in the camera models overrides file.
if data.camera_models_overrides_exists():
    overrides = data.load_camera_models_overrides()
    if "all" in overrides:
        for key in camera_models:
            camera_models[key] = copy.copy(overrides["all"])
            camera_models[key].id = key
    else:
        for key, value in overrides.items():
            camera_models[key] = value
data.save_camera_models(camera_models)

end = time.time()
with open(data.profile_log(), 'a') as fout:
    fout.write('extract_metadata: {0}\n'.format(end - start))



In [5]:
data.config

{'use_exif_size': True,
 'default_focal_prior': 0.9,
 'feature_type': 'HAHOG',
 'feature_root': 1,
 'feature_min_frames': 4000,
 'feature_process_size': 2048,
 'feature_use_adaptive_suppression': False,
 'sift_peak_threshold': 0.1,
 'sift_edge_threshold': 10,
 'surf_hessian_threshold': 3000,
 'surf_n_octaves': 4,
 'surf_n_octavelayers': 2,
 'surf_upright': 0,
 'akaze_omax': 4,
 'akaze_dthreshold': 0.001,
 'akaze_descriptor': 'MSURF',
 'akaze_descriptor_size': 0,
 'akaze_descriptor_channels': 3,
 'akaze_kcontrast_percentile': 0.7,
 'akaze_use_isotropic_diffusion': False,
 'hahog_peak_threshold': 1e-05,
 'hahog_edge_threshold': 10,
 'hahog_normalize_to_uchar': True,
 'lowes_ratio': 0.8,
 'matcher_type': 'FLANN',
 'symmetric_matching': True,
 'flann_algorithm': 'KMEANS',
 'flann_branching': 8,
 'flann_iterations': 10,
 'flann_tree': 8,
 'flann_checks': 20,
 'bow_file': 'bow_hahog_root_uchar_10000.npz',
 'bow_words_to_match': 50,
 'bow_num_checks': 20,
 'bow_matcher_type': 'FLANN',
 'vlad_

# focal length

In [129]:
(1080 / 2) / (math.tan(math.pi*43.60281897/360.0))

1350.0000000923687

In [122]:
2*(30*math.tan(0.5*61.92751306/57.296))

35.99983029809857

In [267]:
37.21 * 1920 / 36

1984.5333333333333

In [ ]:
f_px = f__mm * image_width / sensor_width

In [240]:
61 * 16/9

108.44444444444444

In [239]:
3.99*(8.47)

33.795300000000005

In [238]:
# f_mm to f_px
3.99*(8.47)/36 * 1920

1802.4160000000002

In [237]:
# ratio of sensor widths
36/4.25

8.470588235294118